In [1]:
from pathlib import Path
import joblib
from pprint import pprint
from typing import List, Dict, Tuple, Iterable

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

from pipeline import (
    DropColumns,
    DropHighNAPercentage,
    PreprocessTipoMoneda,
)

In [2]:
ROOT_DIR = Path.cwd().parent
DATA_DIR = ROOT_DIR / "data"
assert DATA_DIR.exists()

DATA_PATH = DATA_DIR / "base_indices_2005-2023.csv"

In [3]:
df = joblib.load(DATA_DIR / "base_indices.pkl")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 218773 entries, 2023-01-01 to 2005-01-01
Data columns (total 59 columns):
 #   Column                                              Non-Null Count   Dtype  
---  ------                                              --------------   -----  
 0   Cód. Institución                                    218773 non-null  int64  
 1   Nombre Institución                                  218773 non-null  object 
 2   Tipo Institución                                    218773 non-null  object 
 3   Clasificación1                                      218773 non-null  object 
 4   Clasificación2                                      218773 non-null  object 
 5   Clasificación3                                      218773 non-null  object 
 6   Clasificación4                                      218773 non-null  object 
 7   Clasificación5                                      218773 non-null  object 
 8   Clasificación6                                  

In [5]:
data_pipeline = Pipeline(
    [
        ("drop_columns", DropColumns()),
        ("drop_high_na", DropHighNAPercentage(na_threshold=0.24)),
        ("preprocess_tipo_moneda", PreprocessTipoMoneda()),
    ]
)

In [6]:
data_pipeline

Pipeline(steps=[('drop_columns', DropColumns()),
                ('drop_high_na', DropHighNAPercentage()),
                ('preprocess_tipo_moneda', PreprocessTipoMoneda())])

In [ ]:
df = data_pipeline.fit_transform(df)

In [ ]:
# Existen columnas que no nos aportan información a nuestro estudio
df = df.drop(
    [
        "Nombre de la Sede",
        "Orden Geográfico de la Región (Norte aSur)",
        "Mención o Especialidad",
        "idgenerocarrera",
        "Cód. Campus",
        "Cód. Sede",
        "Códgo SIES",
    ],
    axis=1,
)

In [ ]:
# También existen columnas con información redundante
# Eliminamos máximos
df = df.drop(
    [
        "Máximo Puntaje (promedio matemáticas y lenguaje)",
        "Máximo Puntaje NEM",
        "Máximo Puntaje Ranking",
    ],
    axis=1,
)

# Eliminamos mínimos
df = df.drop(
    [
        "Mínimo Puntaje (promedio matemáticas y lenguaje)",
        "Mínimo Puntaje NEM",
        "Mínimo Puntaje Ranking",
    ],
    axis=1,
)

In [ ]:
# Otras tienen una cantidad elevada de nulos
vacios = df.isnull().sum()
vacios = (vacios[vacios > 0] / df.shape[0]) * 100
vacios = vacios.astype(int)
vacios = vacios.sort_values(ascending=False)
indices_vacios = vacios[vacios > 24].index

df = df.drop(indices_vacios, axis=1)

In [ ]:
# Por otro lado, podemos notar que el dataframe está desbalanceado
# contamos la cantidad de registros de pregrado vs los de posgrado
df["Pregrado/Posgrado"].value_counts(normalize=True)

In [ ]:
# y que algunas columnas no cuentan con el formato correcto
df["Tipo Moneda"] = df["Tipo Moneda"].str.strip().str.capitalize()
df = df.rename(columns={"Pregrado/Posgrado": "Pregrado o Posgrado"})

In [ ]:
def compute_kmeans(
    df: pd.DataFrame,
    cluster_range: Iterable[int],
) -> pd.DataFrame:
    """
    Perform k-means clustering for a range of k values and collect metrics.
    Args:
        df: Dataframe to cluster.
        cluster_range: Range of k values to try.
    Returns:
        Dictionary with k as key and silhouette score as value.
    """
    interia = []
    silhouette = []
    for k in cluster_range:
        kmeans = KMeans(n_clusters=k, random_state=42).fit(df)
        interia.append(kmeans.inertia_)
        silhouette.append(silhouette_score(df, kmeans.labels_))

    return pd.DataFrame(
        {"inertia": interia, "silhouette": silhouette},
        index=cluster_range,
    )

In [ ]:
def plot_kmean_results(k_range, inertia_values, silhouette_scores):
    """
    Plot the results of the k-means clustering.
    Args:
        k_range: Range of k values used.
        inertia_values: List of inertia values for each k.
        silhouette_scores: List of silhouette scores for each k.
    """
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    ax[0].plot(k_range, inertia_values, marker="o")
    ax[0].set_xlabel("Número de clusters")
    ax[0].set_ylabel("Inercia")
    ax[0].set_title("Método del codo")

    ax[1].plot(k_range, silhouette_scores, marker="o")
    ax[1].set_xlabel("Número de clusters")
    ax[1].set_ylabel("Silhouette")
    ax[1].set_title("Método de la silueta")
    plt.show()

In [ ]:
kmeans_df = compute_kmeans(df, range(2, 21))